In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import *
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image
from keras_vggface import utils

Using TensorFlow backend.


In [5]:
def get_AlexNet():
    inputs = Input(shape=(227,227,3))
    
    # layer 1
    layer1 = Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid')(inputs)
    layer1 = Activation('relu')(layer1)
    layer1 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='valid')(layer1)
    layer1 = BatchNormalization()(layer1)
    
    # layer 2
    layer2_1 = Conv2D(filters=128, kernel_size=(5,5), strides=(2,2), padding='same')(layer1)
    layer2_1 = Activation('relu')(layer2_1)
    layer2_1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid')(layer2_1)
    layer2_1 = BatchNormalization()(layer2_1)

    layer2_2 = Conv2D(filters=128, kernel_size=(5,5), strides=(2,2), padding='same')(layer1)
    layer2_2 = Activation('relu')(layer2_2)
    layer2_2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid')(layer2_2)
    layer2_2 = BatchNormalization()(layer2_2)
    layer2 = concatenate([layer2_1, layer2_2], axis=-1)
    
    # layer 3
    layer3_1 = Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same')(layer2)
    layer3_1 = Activation('relu')(layer3_1)
    
    layer3_2 = Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same')(layer2)
    layer3_2 = Activation('relu')(layer3_2)
    
    # layer 4
    layer4_1 = Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same')(layer3_1)
    layer4_1 = Activation('relu')(layer4_1)

    layer4_2 = Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same')(layer3_2)
    layer4_2 = Activation('relu')(layer4_2)

    # layer 5
    layer5_1 = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same')(layer4_1)
    layer5_1 = Activation('relu')(layer5_1)
    layer5_1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid')(layer5_1)
    layer5_1 = BatchNormalization()(layer5_1)

    layer5_2 = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same')(layer4_2)
    layer5_2 = Activation('relu')(layer5_2)
    layer5_2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid')(layer5_2)
    layer5_2 = BatchNormalization()(layer5_2)
    
    # 5 flat
    layer5_1f = Flatten()(layer5_1)
    layer5_2f = Flatten()(layer5_2)
    layer5_cat = concatenate([layer5_1f, layer5_2f], axis=-1)
    
    # full 1
    f1 = Dense(2048, input_shape=(224*224*3,))(layer5_cat)
    f1 = Activation('relu')(f1)
    f1 = Dropout(0.4)(f1)
    
    f1_2 = Dense(2048, input_shape=(224*224*3,))(layer5_cat)
    f1_2 = Activation('relu')(f1_2)
    f1_2 = Dropout(0.4)(f1_2)
    
    f1_cat = concatenate([f1, f1_2], axis=-1)
    
    # full 2
    f2_1 = Dense(2048)(f1_cat)
    f2_1 = Activation('relu')(f2_1)
    f2_1 = Dropout(0.4)(f2_1)

    f2_2 = Dense(2048)(f1_cat)
    f2_2 = Activation('relu')(f2_2)
    f2_2 = Dropout(0.4)(f2_2)
    
    f2_cat = concatenate([f2_1, f2_2], axis=-1)
    
    # full final
    ff = Dense(1000)(f2_cat)
    ff = Activation('relu')(ff)
#     model.add(Dropout(0.4))

    # extra
#     fe = Dense(22)(ff)
#     fe = Activation('softmax')(fe)
    
    output = ff

    model = Model(inputs, output)
    return model

In [6]:
model = get_AlexNet()

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 55, 55, 96)   34944       input_2[0][0]                    
__________________________________________________________________________________________________
activation_16 (Activation)      (None, 55, 55, 96)   0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 53, 53, 96)   0           activation_16[0][0]              
__________________________________________________________________________________________________
batch_norm